## Finetuning ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli NLI model
- This is based on the Prof. Mihai Surdeanu's text book <a href="https://github.com/clulab/gentlenlp/blob/main/notebooks/chap13_classification_bert.ipynb">Gentle NLP Chapter 13 Classification using BERT model</a>
- Modified for NLI evaluation and analysis over SICCK dataset
- Author: Sushma Anand Akoju, Email: sushmaakoju@arizona.edu

In [1]:
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install 'transformers[torch]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.8 MB/s eta 0:00

# Text Classification Using Transformer Networks (Roberta)

Some initialization:

In [2]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# enable tqdm in pandas
tqdm.pandas()

# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
print(f'device: {device.type}')

# random seed
seed = 1234

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

device: cuda
random seed: 1234


In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
import os
import pandas as pd
path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/final-datasets/block-wise-data/blocks-dec26/data/15/"
assert os.path.exists(path), "Error"
temp = pd.read_csv(os.path.join(path,"SICCK-zero-shot-analysis-apr24.csv"))
df = temp[temp.columns[1:]]
df.head(), len(df)

(   SICK_id                                  Premise  \
 0       90      a man is jumping into an empty pool   
 1       90  every man is jumping into an empty pool   
 2       90      a man is jumping into an empty pool   
 3       90  every man is jumping into an empty pool   
 4       90   some man is jumping into an empty pool   
 
                               Hypothesis Modifier Premise/Hypothesis/Both  \
 0      a man is jumping into a full pool     NONE                    NONE   
 1      a man is jumping into a full pool    every                 Premise   
 2  every man is jumping into a full pool    every              Hypothesis   
 3  every man is jumping into a full pool    every                    Both   
 4      a man is jumping into a full pool     some                 Premise   
 
   Part of Premise/Hypothesis Modified Ground Truth           GT Modifier Type  \
 0                                NONE  Alternation  Alternation  No Modifiers   
 1                          

In [5]:
df['CompressedGT'].unique()

array(['Contradiction', 'Neutral', 'FE', 'RE'], dtype=object)

In [7]:
label2id_roberta = {
    'Contradiction': 2,
    'Neutral': 1,
    'FE': 0,
    'RE': 0,
}

Read the train/dev/test datasets and create a HuggingFace `Dataset` object:

## Load Cross validation sets

In [26]:
filenames = ["fold0.xlsx", "fold1.xlsx", "fold2.xlsx", "fold3.xlsx", "fold4.xlsx"]
path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/june12"
folds = []
columns = ['Premise', 'Hypothesis', 'labels', 'CompressedGT', 'Modifier Type',
           'Modifier',	'Premise/Hypothesis/Both',	'Part of Premise/Hypothesis Modified']
for i,file in enumerate(filenames):
  train = pd.read_excel(os.path.join(path, file), sheet_name="train").rename(columns={"label4roberta":'labels'})
  test = pd.read_excel(os.path.join(path, file), sheet_name="test").rename(columns={"label4roberta":'labels'})
  print(len(train), len(test))
  folds.append({"train":train[columns], "test":test[columns]})
  print(i)

1043 261
0
1043 261
1
1043 261
2
1043 261
3
1043 261
4


### Create data splits with premise, hypothesis as well as hypothesis, premise for **Test** set predictions to label:
- Forward Entailment
- Reverse Entailment
- Neutral

In [28]:
def read_data(data):
    # concatenate title and description, and remove backslashes
    data['text'] = data['Premise'] + " [SEP] " + data['Hypothesis']
    data['text'] = data['text'].str.replace('\\', ' ', regex=False)
    return data

In [29]:
def read_data_reverse(data):
    # concatenate title and description, and remove backslashes
    data['text'] = data['Hypothesis'] + " [SEP] " + data['Premise']
    data['text'] = data['text'].str.replace('\\', ' ', regex=False)
    return data

### Compute metrics for validation and test

In [30]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

def compute_metrics(eval_pred):
    y_true = eval_pred.label_ids
    y_pred = np.argmax(eval_pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(y_true, y_pred), 'recall': recall_score(y_true, y_pred, average='micro'),
            'f1':f1_score(y_true, y_pred, average='micro'), 'precision':precision_score(y_true, y_pred, average='micro')}
def compute_test_metrics(y_true, y_pred):
    return {'accuracy': accuracy_score(y_true, y_pred), 'recall': recall_score(y_true, y_pred, average='micro'),
            'f1':f1_score(y_true, y_pred, average='micro'), 'precision':precision_score(y_true, y_pred, average='micro')}

### To include FE, RE and Neutral label calculation and scores for **Test**

In [31]:
from sklearn.metrics import classification_report
def test_eval(trainer, ds, fold, model_name ):
  test_ds = ds['test'].map(
      tokenize,
      batched=True,
      remove_columns=['Premise', 'Hypothesis', 'text'],
  )
  rev_test_ds = ds['rev_test'].map(
      tokenize,
      batched=True,
      remove_columns=['Premise', 'Hypothesis', 'text'],
  )
  test_ds.to_pandas()
  output = trainer.predict(test_ds)
  rev_scores = trainer.predict(rev_test_ds)

  y_true = output.label_ids
  y_preds = np.argmax(output.predictions, axis=-1)
  y_rev_score_preds = np.argmax(rev_scores.predictions, axis=-1)
  labels = []


  for i in range(len(y_preds)):
        if y_preds[i] == 0:
          labels.append("FE")
        elif y_preds[i] == 2:
          labels.append("Negation")
        else:
          if y_rev_score_preds[i] == 0:
            labels.append("RE")
          else:
            labels.append("Neutral")
  print(classification_report(y_true, y_preds, labels=[0, 1, 2]))

  res = compute_test_metrics(y_true, y_preds)
  res['fold'] = fold
  res['model_name'] = model_name
  return y_true, y_preds, res, labels

In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/nli-deberta-v3-base', num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained('cross-encoder/nli-deberta-v3-base')

### Get this_train, this_validation & this_test set from a this_fold

In [32]:
def get_dataset(fold, model_name):

  columns = ['Premise', 'Hypothesis', 'labels']

  train_df = read_data(fold["train"][columns])
  test_df = read_data(fold["test"][columns])
  rev_test_df = read_data_reverse(fold["test"][columns])
  print(test_df.columns)

  train_df, eval_df = train_test_split(train_df, train_size=0.9)
  train_df.reset_index(inplace=True, drop=True)
  eval_df.reset_index(inplace=True, drop=True)
  test_df.reset_index(inplace=True, drop=True)
  rev_test_df.reset_index(inplace=True, drop=True)

  print(f'train rows: {len(train_df.index):,}')
  print(f'eval rows: {len(eval_df.index):,}')
  print(f'test rows: {len(test_df.index):,}')
  print(f'test rows: {len(rev_test_df.index):,}')

  ds = DatasetDict()
  ds['train'] = Dataset.from_pandas(train_df)
  ds['validation'] = Dataset.from_pandas(eval_df)
  ds['test'] = Dataset.from_pandas(test_df)
  ds['rev_test'] = Dataset.from_pandas(rev_test_df)

  print(ds)
  return ds, test_df, rev_test_df

### CustomTrainer for CrossEntropyLoss but we train for both custom and default Trainer classes in HuggingFace
- Note: we did not see any difference between the two

In [33]:

import torch
from torch import nn
from transformers import Trainer
from accelerate import Accelerator

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss_fct.to('cuda')
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

## Tokenize & Train one model at a time for all folds

In [34]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model_name = ""
model_names =["cross-encoder/nli-deberta-v3-base",	"ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"]
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
output_path = "/content/drive/MyDrive/Colab Notebooks/natural-logic/june12"

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True)

def train(model_name, this_path, folds):
  epochs = [4, 8]
  batch_sizes = [8,16,32]
  m = model_name.split("/")[1]
  all_scores = []
  # tokenizer = AutoTokenizer.from_pretrained(model_name)
  # model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
  # tokenizer = AutoTokenizer.from_pretrained(model_name)
  for num_epochs in epochs:
    for batch_size in batch_sizes:

      for i,fold in enumerate(folds):
          print("\n***********************************************************************************\n")
          print("\n**************** The number of epochs, batch_size and fold respectively are: ",num_epochs, batch_size, i,"************************\n")
          model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

          ds, test_df, rev_test_df = get_dataset(fold,model_name)
          train_ds = ds['train'].map(
            tokenize, batched=True,
            remove_columns=['Premise', 'Hypothesis', 'text'],
          )
          eval_ds = ds['validation'].map(
              tokenize,
              batched=True,
              remove_columns=['Premise', 'Hypothesis', 'text'],
          )

          weight_decay = 0.01
          tx_model_name = f'{model_name}-sequence-classification'

          training_args = TrainingArguments(
              output_dir=os.path.join(output_path,m+"_"+str(num_epochs)+str(batch_size)+"trainer"),
              log_level='error',
              num_train_epochs=num_epochs,
              per_device_train_batch_size=batch_size,
              per_device_eval_batch_size=batch_size,
              evaluation_strategy='epoch',
              weight_decay=weight_decay,
          )
          trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            compute_metrics=compute_metrics,
            tokenizer=tokenizer,
          )
          trainer.train()

          customTrainer  = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=train_ds,
            eval_dataset=eval_ds,
            compute_metrics=compute_metrics,
            tokenizer=tokenizer,
          )

          customTrainer.train()
          y_true, y_pred, results, labels = test_eval(trainer, ds, i, model_name )
          y_true1, y_pred1, results1, labels1 = test_eval(customTrainer, ds, i, model_name )

          all_scores.append(results)
          fold["test"]["label"]= y_true
          fold["test"]["predictions"] = y_pred
          fold["test"]["predictions2"] = y_pred1
          fold["test"]["text"] = test_df['text']
          fold["test"]["pred_labels"] =  labels
          filename = "five_"+m+"_"+str(num_epochs)+"_"+str(batch_size)+"_"+str(i)+"_test.xlsx"
          fold["test"].to_csv(os.path.join(this_path, filename))
  return all_scores

In [35]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
# if tokenizer:
#   del tokenizer
# if model:
#   del model

### "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"

In [ ]:
all_scores = []
predictions = []
model_name = model_names[1]
m = model_name.split("/")[1]
this_path = os.path.join(path, m)
if not os.path.exists(this_path):
  os.mkdir(this_path)
assert os.path.exists(this_path), "%s Path does not exists!"%(this_path)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)
all_scores.append(train(model_name, this_path, folds))


***********************************************************************************


**************** The number of epochs, batch_size and fold respectively are:  4 8 0 ************************

Index(['Premise', 'Hypothesis', 'labels', 'text'], dtype='object')
train rows: 938
eval rows: 105
test rows: 261
test rows: 261
DatasetDict({
    train: Dataset({
        features: ['Premise', 'Hypothesis', 'labels', 'text'],
        num_rows: 938
    })
    validation: Dataset({
        features: ['Premise', 'Hypothesis', 'labels', 'text'],
        num_rows: 105
    })
    test: Dataset({
        features: ['Premise', 'Hypothesis', 'labels', 'text'],
        num_rows: 261
    })
    rev_test: Dataset({
        features: ['Premise', 'Hypothesis', 'labels', 'text'],
        num_rows: 261
    })
})


<ipython-input-28-3e308bfb0585>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['Premise'] + " [SEP] " + data['Hypothesis']
<ipython-input-28-3e308bfb0585>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].str.replace('\\', ' ', regex=False)
<ipython-input-28-3e308bfb0585>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Map:   0%|          | 0/938 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

<ipython-input-28-3e308bfb0585>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['Premise'] + " [SEP] " + data['Hypothesis']
<ipython-input-28-3e308bfb0585>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].str.replace('\\', ' ', regex=False)
<ipython-input-28-3e308bfb0585>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Map:   0%|          | 0/938 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'eval_loss': 0.3832259178161621, 'eval_accuracy': 0.9142857142857143, 'eval_recall': 0.9142857142857143, 'eval_f1': 0.9142857142857143, 'eval_precision': 0.9142857142857143, 'eval_runtime': 0.6943, 'eval_samples_per_second': 151.232, 'eval_steps_per_second': 10.082, 'epoch': 1.0}
{'eval_loss': 0.35512474179267883, 'eval_accuracy': 0.9142857142857143, 'eval_recall': 0.9142857142857143, 'eval_f1': 0.9142857142857143, 'eval_precision': 0.9142857142857143, 'eval_runtime': 0.689, 'eval_samples_per_second': 152.386, 'eval_steps_per_second': 10.159, 'epoch': 2.0}
{'eval_loss': 0.20690137147903442, 'eval_accuracy': 0.9523809523809523, 'eval_recall': 0.9523809523809523, 'eval_f1': 0.9523809523809523, 'eval_precision': 0.9523809523809523, 'eval_runtime': 0.6834, 'eval_samples_per_second': 153.647, 'eval_steps_per_second': 10.243, 'epoch': 3.0}
{'eval_loss': 0.32045701146125793, 'eval_accuracy': 0.9619047619047619, 'eval_recall': 0.9619047619047619, 'eval_f1': 0.9619047619047619, 'eval_precision

In [ ]:
all_scores_deberta = all_scores

In [ ]:
all_scores_deberta = all_scores